# [모듈 5.1] 모델 빌딩 파이프라인 개발 (SageMaker Model Building Pipeline 모든 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 30분** 소요 됩니다.

- 0. SageMaker Model Building Pipeline 개요
- 1. 파이프라인 변수 및 환경 설정
- 2. 파이프라인 스텝 단계 정의

    - (1) 전처리 스텝 스텝 정의    
    - (2) 모델 학습을 위한 학습 스텝 정의 
    - (3) 모델 평가 스텝
    - (4) 모델 등록 스텝
    - (5) HPO 단계    
    - (6) 최고 모델 등록 스텝    
    - (6) HPO 스텝
    - (7) 조건 스텝
    
- 3. 모델 빌딩 파이프라인 정의 및 실행
- 4. Pipleline 캐싱 및 파라미터 이용한 실행
- 5. 정리 작업
    
---

# 0.SageMaker Model Building Pipeline 개요

Amazon SageMaker 모델 구축 파이프라인은 직접 SageMaker 통합을 활용하는 머신 러닝 파이프라인을 구축하기 위한 도구입니다. 이러한 통합으로 인해 많은 단계 생성 및 관리를 처리하는 도구를 사용하여 파이프라인을 생성하고 오케스트레이션용 SageMaker Projects를 설정할 수 있습니다. SageMaker 파이프라인은 다른 파이프라인에 비해 다음과 같은 이점을 제공합니다

- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)

# 1. 파이프라인 변수 및 환경 설정



In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

%store -r 

## (1) 모델 빌딩 파이프라인 변수 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 프로세싱 스텝을 위한 인스턴스 타입 및 인스턴스 수
    - 데이터 전처리 스텝 및 실시간 앤드 포인트 스텝에 사용 됨.
- 훈련 스텝을 위한 인스턴스 타입 및 인스턴스 수    
- 모델 평가를 통해 나온 validation:roc-auc 에 대한 분기 조건 값
- 원본 데이터 세트에 대한 S3 주소
    - 데이터 전처리 스텝에서 사용 됩니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [2]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)

model_eval_threshold = ParameterFloat(
    name="model2eval2threshold",
    default_value=0.85
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


## 캐싱 정의

- 참고: 캐싱 파이프라인 단계:  [Caching Pipeline Steps](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)

In [3]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, 
                           expire_after="7d")


# 2. 파이프라인 스텝 단계 정의

## (1) 전처리 스텝 단계 정의
- input_data_uri 입력 데이타를 대상으로 전처리를 수행 합니다.

크게 아래와 같은 순서로 정의 합니다.
- 프로세싱 오브젝트 정의 (SKLearnProcessor)
- 프로세싱 스텝 정의
    - 일력 데이터 세트
        - source: S3 경로 (input_data_uri)
        - destination: 도커 컨테이너의 내부 폴더 위치
    - 출력 위치
        - 훈련 전처리 데이터 결과 위치
        - 테스트 전처리 데이터 결과 위치
    - 프로세싱 코드
    - 프로세싱 코드에 넘길 인자 


In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor

processing_instance_type ='ml.m5.xlarge'
split_rate = 0.2
framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)

# print("input_data: \n", input_data)

In [7]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    
step_process = ProcessingStep(
    name="FraudProcess",
    processor=sklearn_processor,
    inputs=[
#         ProcessingInput(source=input_data_uri,destination='/opt/ml/processing/input'),
        ProcessingInput(source=input_data, destination='/opt/ml/processing/input'),        
         ],
    outputs=[ProcessingOutput(output_name="train",
                              source='/opt/ml/processing/output/train'),
             ProcessingOutput(output_name="test",
                              source='/opt/ml/processing/output/test')],
    job_arguments=["--split_rate", f"{split_rate}"],        
    code= 'src/preprocessing.py',
    cache_config = cache_config, # 캐시 정의
)


## (2) 모델 학습을 위한 학습단계 정의 

학습 스텝을 정의하기 위해서는 크게 아래와 같은 과정이 있습니다.
- XGBoost Estimator 정의
- 학습 스텝 정의
    - 아래와 같은 중요한 인자가 필요 합니다.
        - Estimator (위에서 정의한 것 사용)
        - 훈련을 위한 입력 데이터 위치


### 기본 훈련 변수 및 하이퍼파라미터 설정

In [8]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = 'fraud2train'
estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'

base_hyperparameters = {
       "scale_pos_weight" : "29",        
        "max_depth": "6",
        "alpha" : "0", 
        "eta": "0.3",
        "min_child_weight": "1",
        "objective": "binary:logistic",
        "num_round": "100",
}


In [10]:
training_instance_type ='ml.m5.xlarge'

xgb_train = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = base_hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

훈련의 입력이 이전 전처리의 결과가 제공됩니다.
- `step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri`

In [11]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            # s3_data= train_preproc_dir_artifact,            
            content_type="text/csv"
        ),
    },
    cache_config = cache_config, # 캐시 정의    
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/workflow/steps.py:391: UserWarning: Profiling is enabled on the provided estimator. The default profiler rule includes a timestamp which will change each time the pipeline is upserted, causing cache misses. If profiling is not needed, set disable_profiler to True on the estimator.
  warnings.warn(msg)


## (3) 모델 평가 단계

### SKLearnProcessor 의 기본 도커 컨테이너 지정
ScriptProcessor 의 기본 도커 컨테이너로 Scikit-learn를 기본 이미지를 사용함. 
- 사용자가 정의한 도커 컨테이너도 사용할 수 있습니다.

In [12]:
from sagemaker.processing import ScriptProcessor

script_eval = SKLearnProcessor(
                             framework_version= "0.23-1",
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=1,
                             base_job_name="script-fraud-scratch-eval",
                                    )


모델 평가 지표를 jons 파일에 저장을 하고 property로 등록을 합니다.

In [13]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

step_eval = ProcessingStep(
    name="FraudEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source= step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
        destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="src/evaluation.py",
    cache_config = cache_config, # 캐시 정의    
  property_files=[evaluation_report], 
)

## (4) 모델 등록 스텝

### 모델 그룹 생성
- 위의 step_eval 에서 S3 로 올린 evaluation.json 파일안의 지표를 "모델 레지스트리" 안에 모델 패키지의 모델 버전 등록시에 삽입함

- 참고
    - 모델 그룹 릭스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)
    - 모델 지표 등록: [Model Quality Metrics](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)

In [14]:
model_package_group_name = f"{project_prefix}"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}
response = sm_client.list_model_package_groups(NameContains=model_package_group_name)
if len(response['ModelPackageGroupSummaryList']) == 0:
    print("No model group exists")
    print("Create model group")    
    
    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))    
else:
    print(f"{model_package_group_name} exitss")

No model group exists
Create model group
ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:057716757052:model-package-group/sagemaker-pipeline-step-by-step-phase02


In [15]:
from sagemaker.workflow.step_collections import RegisterModel

from sagemaker.model_metrics import MetricsSource, ModelMetrics 

# 위의 step_eval 에서 S3 로 올린 evaluation.json 파일안의 지표를 "모델 레지스트리" 에 모데 버전 등록시에 삽입함
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)


step_register = RegisterModel(
    name= "FraudcRegisterhModel",
    estimator=xgb_train,
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

## (5) HPO 스텝
- 하이퍼 파라미터 튜닝 스텝을 정의 합니다.

In [16]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}


objective_metric_name = "validation:auc"

tuner = HyperparameterTuner(
    xgb_train, objective_metric_name, hyperparameter_ranges, 
    max_jobs=5,
    max_parallel_jobs=5,
)

from sagemaker.workflow.steps import TuningStep
    
step_tuning = TuningStep(
    name = "FraudTuning",
    tuner = tuner,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            # s3_data= train_preproc_dir_artifact,            
            content_type="text/csv"
        ),
    },    
    cache_config = cache_config, # 캐시 정의        
)

## (6) 최고 모델 등록 스텝

### 최고의 모델 생성 및 등록

하이퍼파라미터 튜닝 작업을 성공적으로 완료한 후TuningStep의 교육 작업에 의해 생성된 모델 아티팩트에서 SageMaker 모델을 생성하거나 모델 레지스트리에 모델을 등록할 수 있습니다.


TuningStep 클래스의 get_top_model_s3_uri 메서드를 사용하여 최고 성능의 모델 버전의 모델 아티팩트를 가져옵니다.

- `model_bucket_key` 는 튜닝 스텝을 통해서 생성된 훈련 잡의 위치 입니다.

In [17]:
model_bucket_key = estimator_output_path.split('//')[1]
print("model_bucket_key: ", model_bucket_key)
print("model_package_group_name: ", model_package_group_name)

model_bucket_key:  sagemaker-us-east-1-057716757052/fraud2train/training_jobs
model_package_group_name:  sagemaker-pipeline-step-by-step-phase02


In [18]:
from sagemaker.workflow.step_collections import RegisterModel


step_register_best = RegisterModel(
    name="RegisterBestFraudModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_bucket_key),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)

## (7) 조건 스텝

In [19]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value",
    ),
    # right=8.0
    right = model_eval_threshold
)

step_cond = ConditionStep(
    name="FruadMetricCond",
    conditions=[cond_lte],
    if_steps=[step_tuning],        
    else_steps=[step_register], 
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# 3.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.
-     steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 20분 정도 소요 됩니다. 이후에 스튜디오에서 아래와 같이 확인이 가능합니다.

- ![all-pipeline-phase02-wo-cache.png](img/all-pipeline-phase02-wo-cache.png)

In [20]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'sagemaker-pipeline-phase2-mbp-step-by-step'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        training_instance_count,                
        input_data,
        model_eval_threshold,
        model_approval_status,        
    ],

    
  steps=[step_process, step_train, step_eval, step_cond, step_register_best],
)



In [21]:
import json

definition = json.loads(pipeline.definition())
# definition

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


### 파이프라인을 SageMaker에 제출하고 실행하기 


In [22]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/sagemaker-pipeline-phase2-mbp-step-by-step',
 'ResponseMetadata': {'RequestId': 'b03c7072-69ab-44bf-b871-12b3c2fc3a4a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b03c7072-69ab-44bf-b871-12b3c2fc3a4a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Thu, 23 Jun 2022 11:27:03 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [23]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [24]:
execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [25]:
execution.list_steps()

[{'StepName': 'RegisterBestFraudModel',
  'StartTime': datetime.datetime(2022, 6, 23, 11, 42, 48, 339000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 11, 42, 49, 607000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/1'}}},
 {'StepName': 'FraudTuning',
  'StartTime': datetime.datetime(2022, 6, 23, 11, 38, 59, 566000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 11, 42, 47, 391000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TuningJob': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:hyper-parameter-tuning-job/brnj2j2p3jud-fraudtu-4dfdzcsaa2'}}},
 {'StepName': 'FruadMetricCond',
  'StartTime': datetime.datetime(2022, 6, 23, 11, 38, 58, 750000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 11, 38, 59, 144000, tzinfo=tzlocal()),
  'StepSt

# 4. Pipeline 캐싱 및 파라미터 이용한 실행
- 캐싱은 2021년 7월 현재 Training, Processing, Transform 의 Step에 적용이 되어 있습니다.
- 상세 사항은 여기를 확인하세요. -->  [캐싱 파이프라인 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)

- 아래를 실행하면 아래와 같이 스튜디오에서 확인이 가능합니다.
- ![all-pipeline-phase02-cache.png](img/all-pipeline-phase02-cache.png)


In [26]:
is_cache = True

In [27]:
%%time 

from IPython.display import display as dp
import time

if is_cache:
    execution = pipeline.start(
        parameters=dict(
            model2eval2threshold=0.8,
        )
    )    
    
    # execution = pipeline.start()
    execution.wait()


CPU times: user 18.8 ms, sys: 1.11 ms, total: 19.9 ms
Wall time: 30.9 s


In [28]:
if is_cache:
    dp(execution.list_steps())

[{'StepName': 'FraudcRegisterhModel',
  'StartTime': datetime.datetime(2022, 6, 23, 11, 43, 19, 882000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 11, 43, 20, 531000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/sagemaker-pipeline-step-by-step-phase02/2'}}},
 {'StepName': 'FruadMetricCond',
  'StartTime': datetime.datetime(2022, 6, 23, 11, 43, 19, 167000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 11, 43, 19, 626000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Condition': {'Outcome': 'False'}}},
 {'StepName': 'FraudEval',
  'StartTime': datetime.datetime(2022, 6, 23, 11, 43, 18, 118000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 6, 23, 11, 43, 18, 661000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'CacheHitResult': {'SourcePipelineExecutionArn': 'arn:aws:sagemaker:us-e

# 5. 테스트 데이타 추출

- 추후에 모델 배포 후 추론을 위해서 전처리된 테스트 데이타를 추출 합니다.


In [29]:
def get_proc_artifact(execution, client, kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()

    proc_arn = response[-1]['Metadata']['ProcessingJob']['Arn'] # index -1은 가장 처음 실행 step
    #proc_arn = response[-1]['Metadata']
    # print("proc_arn: ", proc_arn)
    proc_job_name = proc_arn.split('/')[-1]
    #print("proc_job_name: ", proc_job_name)
    
    response = client.describe_processing_job(ProcessingJobName = proc_job_name)
    test_preprocessed_file = response['ProcessingOutputConfig']['Outputs'][kind]['S3Output']['S3Uri'] # index 1: test 파일    
    print("test_preprocessed_file: \n ", test_preprocessed_file)
    
    return test_preprocessed_file

import boto3
client = boto3.client("sagemaker")

test_preproc_dir_artifact = get_proc_artifact(execution, client, kind=1 )



test_preprocessed_file: 
  s3://sagemaker-us-east-1-057716757052/FraudProcess-9eb3e47661f2f1f3db35bb21017985f2/output/test


## 변수 저장

In [30]:
all_mbp_pipeline_name = pipeline_name # 파이프라인 변수 이름을 변경
%store all_mbp_pipeline_name
%store test_preproc_dir_artifact

Stored 'all_mbp_pipeline_name' (str)
Stored 'test_preproc_dir_artifact' (str)
